In [12]:
import pandas as pd
import numpy as np
import os, os.path
import re

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [13]:
not_inc_regions = [ 
                    'армавир',
                    'амурская область',
                    'архангельсая область без ао',
                    'архангельская область',
                    'архангельская область без авт. округа',
                    'архангельская область без авт.округа',
                    'архангельская область без автономного округа',
                    'архангельская область без ао',
                    'архангельская область без нао',
                    'архангельская область кроме ненецкого автономного округа',
                    'астраханская область',
                    'белская область',
                    'в том числе: ненецкий автономный округ',
                    'владимирская область',
                    'вологда',
                    'дагестан',
                    'дальневосточный',
                    'еврейская авт.область',
                    'еврейская автономная область',
                    'забайкальский край',
                    'ингушетия',
                    'иркутская область',
                    'кабардино-балкарская',
                    'калмыкия',
                    'калужская область',
                    'камчатский край',
                    'карачаево-черкесская',
                    'краснодар',
                    'крым',
                    'курганская область',
                    'магаданская область',
                    'марий эл',
                    'мурманская область',
                    'ненецкий авт.округ',
                    'ненецкий автономный округ',
                    'ненецкий автономный округ архангельская область',
                    'ненецкий ао',
                    'новороссийск',
                    'новская область',
                    'оренбургская область',
                    'отгружено товаров собственного производства, выполнено работ и услуг собственными силами малыми предприятиями включая микропредприятия',
                    'приволжский',
                    'псковская область',
                    'рязанская область',
                    'саха якутия',
                    'сахалинская область',
                    'севастополь',
                    'северная осетия - алания',
                    'северная осетия- алания',
                    'северная осетия-алания',
                    'северо-западный',
                    'северо-кавказский',
                    'сибирский',
                    'сочи',
                    'средняя численность работников малых предприятий без микропредприятий',
                    'средняя численность работников микропредприятий',
                    'тамбовская область',
                    'тверская область',
                    'томск',
                    'уральский',
                    'хабаровский край',
                    'хакасия',
                    'череповец',
                    'чеченская',
                    'чита',
                    'чувашская',
                    'чувашская - чувашия',
                    'чувашская -чувашия',
                    'чукотский авт.округ',
                    'чукотский автономный округ',
                    'южный',
                    'ямало-ненецкий авт.округ',
                    'ямало-ненецкий автономный округ',
                    'ямало-ненецкий автономный округ тюменская область',
                    'ямало-ненецкий ао',
                    'nan'
]

In [14]:
def replaces(data):
    map_repl = {'город федерального значения': '',
                'город': '',
                ')': '',
                '(': '',
                'h': 'н',
                'k': 'к',
                'республика': '',
                'г.': '',
               'столица российской федерации': ''}
    for old_value, new_value in map_repl.items():
        data['region'] = data['region'].apply(lambda x: str(x).lower().replace(old_value, new_value).strip())
    data['region'] = data['region'].apply(lambda x: re.sub(r'\s+', ' ', x))
    return data

In [15]:
def preprocess_data(data):
    for col in data.columns[1:]:
        data = replaces(data)
        data[col] = data[col].apply(lambda x: str(x).strip().replace('-', 'nan').replace(' ', '').replace(',', '.')).astype('float')
        data = data[data['region'].apply(lambda x: 'федерация' not in x and 'федеральный округ' not in x)]
        data = data.iloc[:-1]

        regions_for_repl = {
            'тюменская область без ао': 'тюменская область',
            'удмуртская': 'удмуртия',
            'ханты-мансийский ао-югра': 'ханты-мансийский ао',
            'кемеровская область - кузбасс': 'кемеровская область',
            'адыгея адыгея': 'адыгея',
            'тюменская область без автономных округов': 'тюменская область',
            'тюменская область без авт. округов': 'тюменская область',
            'белская область': 'белгородская область',
            'нижеская область': 'нижегородская область',
            'татарстан татарстан': 'татарстан',
            'тюменская область без авт.округов': 'тюменская область'
        }
        
        data.loc[data['region'].apply(lambda x: 'ханты' in x), 'region'] = 'ханты-мансийский ао'
        
        for old_region_name, new_region_name in regions_for_repl.items():
            data['region'] = data['region'].apply(lambda x: x.replace(old_region_name, new_region_name))
        
    return data

In [16]:
def merge_data(file_paths):
    for i, path in enumerate(file_paths):

        data = pd.read_csv(path, sep='\t', encoding='windows-1251')
        data = preprocess_data(data)
        
        if i == 0:
            merged_data = data.copy()
        elif i != 0:
            merged_data = pd.merge(merged_data, data, how='outer', on='region')
            
    cols = [col for col in merged_data.columns if 'Unnamed' not in col]
    
    return merged_data[cols]

In [17]:
file_paths = [os.path.join('../', 'add_data', file_name) for file_name in os.listdir(os.path.join('../', 'add_data'))]

merged_data = merge_data(file_paths)

In [18]:
print(merged_data.shape)

for region in not_inc_regions:
    merged_data = merged_data[merged_data['region'] != region].copy()
    
print(merged_data.shape)

merged_data.drop_duplicates('region', keep='last', inplace=True)

print(merged_data.shape)

(196761, 29)
(163918, 29)
(49, 29)


In [19]:
cols_with_nulls = merged_data.isnull().sum()
cols_with_nulls = cols_with_nulls[cols_with_nulls > 0].index

for col in cols_with_nulls:
    merged_data.fillna(merged_data[col].median(), inplace=True)
    
merged_data.reset_index(inplace=True, drop=True)

In [21]:
merged_data

,region,Доля городского населения в общей численности,Оборот малых предприятий (без микропредприятий),Оборот малых предприятий (включая микропредприятия),Оборот микропредприятий,Оборот средних организаций,"Объем выручки от продажи товаров, продукции, работ, услуг",Отгруженные товары собственного производства в рублях,Отгружено товаров собственного производства малыми предприятиями (включая микропредприятия),Продано товаров несобственного производства микропредприятиями,"Домохозяйства, проживающие в городской местности","Домохозяйства, проживающие в сельской местности","Реальные денежные доходы населения по субъектам Российской Федерации, в % к соответствующему периоду",Среднедушевые денежные доходы населения,Средняя численность работников малых предприятий (без микропредприятий),Средняя численность работников микропредприятий,Оплата труда,Доходы от предпринимательской деятельности,Социальные выплаты,Доходы от собственности,Прочие денежные поступления,Денежные доходы в расчете на душу населения,Место в Российской Федерации,Численность безработных в возрасте 15-72 лет,Численность занятых в возрасте 15-72 лет,Численность постоянного населения,Численность индивидуальных предпринимателей,Число выбывших из региона,Число прибывших в регион
0,белгородская область,67.50,2.783802e+08,5.619534e+08,2.755572e+08,91.289907,2.018216e+08,1.022596e+08,2.294508e+08,1.732976e+08,30823.7,24586.6,98.1,32835.0,55501.0,49759.0,45.2,9.7,20.1,3.4,21.6,32835.0,23.0,40.583,791.520,1548284.0,31463.0,21452.0,22254.0
1,брянская область,70.40,1.280093e+08,2.626825e+08,1.244798e+08,36.492223,1.485398e+08,4.315935e+07,1.115036e+08,8.132044e+07,20899.5,20307.6,95.3,28310.0,36862.0,29997.0,42.2,7.0,23.5,2.7,24.6,28310.0,43.0,23.287,559.530,1196339.0,19614.0,18864.0,18579.0
2,воронежская область,68.00,4.099698e+08,7.684007e+08,3.311705e+08,135.670014,3.131752e+08,1.386809e+08,3.287951e+08,1.924896e+08,24426.0,20045.8,95.2,31997.0,90935.0,79018.0,45.4,5.8,19.5,3.4,25.9,31997.0,28.0,50.370,1116.669,2326013.0,41933.0,31156.0,31504.0
3,ивановская область,81.70,1.427356e+08,3.674473e+08,2.245467e+08,44.506854,1.169242e+08,9.655219e+07,1.655579e+08,1.279945e+08,29301.8,21049.3,97.0,26219.0,41309.0,36378.0,49.9,6.8,23.8,3.5,16.0,26219.0,54.0,27.887,485.698,1000657.0,20563.0,18846.0,17978.0
4,костромская область,72.70,7.508434e+07,1.327835e+08,6.624659e+07,33.257568,8.031330e+07,3.691790e+07,6.990199e+07,2.932869e+07,23275.1,21505.4,95.4,25260.0,25788.0,21373.0,55.9,9.8,26.4,3.1,4.8,25260.0,62.0,17.057,292.038,635325.0,12582.0,11502.0,12884.0
5,курская область,68.50,1.105906e+08,2.589581e+08,1.149733e+08,51.698682,1.416654e+08,4.117116e+07,1.148470e+08,7.380211e+07,27234.2,23383.8,97.4,29671.0,30898.0,24036.0,47.8,10.4,23.0,2.9,15.9,29671.0,35.0,27.098,529.396,1105524.0,21700.0,16062.0,17114.0
6,липецкая область,64.59,1.329162e+08,2.858238e+08,1.444314e+08,37.911230,1.553607e+08,5.639557e+07,1.281382e+08,8.803583e+07,24421.5,25438.5,94.9,32201.0,36329.0,32566.0,46.9,11.5,20.2,3.3,18.1,32201.0,25.0,25.770,569.870,1141703.0,24299.0,12913.0,12135.0
7,московская область,81.40,1.733147e+09,2.907394e+09,9.222480e+08,662.932350,8.455883e+08,4.387859e+08,1.338049e+09,4.834621e+08,37362.5,20252.7,96.7,46519.0,330696.0,291026.0,52.7,4.5,13.7,5.0,24.1,46519.0,10.0,149.592,3997.307,7645255.0,173543.0,198600.0,212965.0
8,орловская область,66.80,7.663261e+07,1.457719e+08,7.199245e+07,31.812906,6.790706e+07,3.044362e+07,6.632338e+07,4.154883e+07,28984.2,32366.1,98.0,26705.0,23258.0,17129.0,52.6,11.6,27.5,2.6,5.7,26705.0,52.0,21.314,326.918,736483.0,15869.0,8695.0,6815.0
9,смоленская область,71.80,1.546308e+08,3.173210e+08,1.445269e+08,55.612370,9.299903e+07,4.332619e+07,1.087075e+08,1.012007e+08,21155.1,14458.3,97.8,27998.0,35356.0,31028.0,51.8,3.8,23.4,3.9,17.1,27998.0,44.0,25.210,451.226,938625.0,18409.0,15669.0,14432.0


In [37]:
train_data = pd.read_csv('../data/data/train.csv')
test_data = pd.read_csv('../data/data/test.csv')
train_data['region'] = train_data['region'].apply(lambda x: x.lower())

In [38]:
new_train_data = pd.merge(train_data, merged_data, on='region', how='left')

In [39]:
new_train_data

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,...,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type,Доля городского населения в общей численности,Оборот малых предприятий (без микропредприятий),Оборот малых предприятий (включая микропредприятия),Оборот микропредприятий,Оборот средних организаций,"Объем выручки от продажи товаров, продукции, работ, услуг",Отгруженные товары собственного производства в рублях,Отгружено товаров собственного производства малыми предприятиями (включая микропредприятия),Продано товаров несобственного производства микропредприятиями,"Домохозяйства, проживающие в городской местности","Домохозяйства, проживающие в сельской местности","Реальные денежные доходы населения по субъектам Российской Федерации, в % к соответствующему периоду",Среднедушевые денежные доходы населения,Средняя численность работников малых предприятий (без микропредприятий),Средняя численность работников микропредприятий,Оплата труда,Доходы от предпринимательской деятельности,Социальные выплаты,Доходы от собственности,Прочие денежные поступления,Денежные доходы в расчете на душу населения,Место в Российской Федерации,Численность безработных в возрасте 15-72 лет,Численность занятых в возрасте 15-72 лет,Численность постоянного населения,Численность индивидуальных предпринимателей,Число выбывших из региона,Число прибывших в регион
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,0,0,0,0,2,4,6,3.293470,Пермь,1055397.0,0.027732,3,6,17,34,0,0,1,1,0,0,1,2,2,3,4,0,0,1,0,0,0,0,1,2,0,1,2,4,4,...,0,0,0,0.002864,4,13,21,139937.500000,136,49,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,пермский край,32.000000,S27289,2020-01-05,10,0,75.90,3.660218e+08,7.400123e+08,3.622188e+08,102.637486,3.717217e+08,1.886904e+08,3.543473e+08,1.735284e+08,30971.7,21563.2,94.3,29937.0,92040.0,87700.0,55.2,6.8,20.5,3.7,13.8,29937.0,33.0,70.496,1159.579,2605030.0,46281.0,42259.0,38872.0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,0,0,1,0,2,2,6,43.950989,Орехово-Зуево,120184.0,0.089441,0,31,50,57,0,1,2,3,0,0,1,2,1,1,3,2,4,6,2,2,2,1,1,2,0,1,2,3,1,...,0,0,0,0.154661,4,10,11,60410.714286,146,37,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,московская область,280.000000,S17052,2020-01-05,10,0,81.40,1.733147e+09,2.907394e+09,9.222480e+08,662.9323